In [1]:
from ogb.nodeproppred import NodePropPredDataset
from ogb.graphproppred import GraphPropPredDataset
from sklearn.model_selection import ParameterGrid
%reload_ext autoreload
%autoreload 2
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import convo as c 
import convL 
import modularized as m 
import batching as b 
import poolingL
import gin
import math 
import funcy as fy
import gridSearch as g 
import json 
import os



/home/olga/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-16 15:52:32.321386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 15:52:34.778554: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-16 15:52:34.778864: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [37]:
import sys
sys.path.append("/content/drive/Othercomputers/Mein PC/Completed")

In [2]:
dataset1 = GraphPropPredDataset(name = 'ogbg-molhiv')

In [7]:
dataset_arxiv = NodePropPredDataset(name = 'ogbn-arxiv')

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:31<00:00,  2.60it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 3778.65it/s]

Saving...


In [2]:
dataset_proteins = NodePropPredDataset(name='ogbn-proteins')


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:05<00:00,  5.63s/it]


Saving...


: 

: 

[[27]]
[[34]]
[[28]]
[[3]]
[[28]]
[[16]]
[[6]]
[[27]]
[[8]]
[[28]]
[[27]]
[[0]]
[[28]]
[[28]]
[[34]]
[[8]]
[[16]]
[[34]]
[[27]]
[[19]]
[[28]]
[[16]]
[[28]]
[[28]]
[[28]]
[[8]]
[[16]]
[[28]]
[[28]]
[[8]]
[[27]]
[[34]]
[[24]]
[[3]]
[[34]]
[[28]]
[[4]]
[[27]]
[[34]]
[[34]]
[[8]]
[[3]]
[[28]]
[[28]]
[[28]]
[[28]]
[[19]]
[[28]]
[[24]]
[[28]]
[[28]]
[[16]]
[[16]]
[[27]]
[[28]]
[[34]]
[[34]]
[[28]]
[[27]]
[[28]]
[[28]]
[[28]]
[[3]]
[[34]]
[[34]]
[[28]]
[[28]]
[[16]]
[[28]]
[[34]]
[[8]]
[[28]]
[[28]]
[[26]]
[[0]]
[[34]]
[[34]]
[[22]]
[[28]]
[[28]]
[[28]]
[[0]]
[[19]]
[[8]]
[[27]]
[[27]]
[[28]]
[[36]]
[[16]]
[[3]]
[[28]]
[[34]]
[[16]]
[[8]]
[[27]]
[[28]]
[[4]]
[[34]]
[[27]]
[[28]]
[[6]]
[[24]]
[[16]]
[[8]]
[[24]]
[[28]]
[[16]]
[[28]]
[[28]]
[[3]]
[[28]]
[[28]]
[[27]]
[[28]]
[[28]]
[[28]]
[[19]]
[[34]]
[[28]]
[[34]]
[[3]]
[[3]]
[[28]]
[[34]]
[[28]]
[[27]]
[[16]]
[[34]]
[[34]]
[[24]]
[[16]]
[[28]]
[[28]]
[[3]]
[[27]]
[[16]]
[[28]]
[[0]]
[[16]]
[[28]]
[[28]]
[[16]]
[[19]]
[[19]]
[[36]]
[[16]]
[[24

In [3]:
param_grid = {'num_layers': [1, 2, 3, 4, 5, 6],
              'learning_rate': [0.0001, 0.001, 0.01],
              'optimizer': ['Adam'],
              'regularization': ['DropOut', 'NodeSampling', 'DropEdge', 'GDC'],
              'probability': [0.3, 0.5, 0.7],
              'activation': ['relu', 'sigmoid', 'tanh'],
              'units': [32, 64],
              'convo_type': ['gcn', 'gin']}

override_conv_type = 'gin' # None

if override_conv_type is not None:
    param_grid['convo_type'] = [override_conv_type]

small_sample = list(ParameterGrid(param_grid))[0:1]
sample = list(ParameterGrid(param_grid))[0:5]
big_sample = list(ParameterGrid(param_grid))[0:20]
small_sample

[{'activation': 'relu',
  'convo_type': 'gin',
  'learning_rate': 0.0001,
  'num_layers': 1,
  'optimizer': 'Adam',
  'probability': 0.3,
  'regularization': 'DropOut',
  'units': 32}]

In [6]:
g.create_grid_models(sample,'ogbg-molhiv')

[0] Evaluating hparams: {'activation': 'relu', 'convo_type': 'gin', 'learning_rate': 0.0001, 'num_layers': 1, 'optimizer': 'Adam', 'probability': 0.3, 'regularization': 'DropOut', 'units': 32}
Epoch 1/3
138/138 [==============================] - 6s 22ms/step - loss: 0.5236 - auc_2: 0.5801 - binary_accuracy: 0.9220 - precision_2: 0.0474 - recall_2: 0.0769 - val_loss: 0.2145 - val_auc_2: 0.5000 - val_binary_accuracy: 0.9684 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/3
138/138 [==============================] - 3s 22ms/step - loss: 0.1602 - auc_2: 0.5000 - binary_accuracy: 0.9684 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - val_loss: 0.1429 - val_auc_2: 0.5000 - val_binary_accuracy: 0.9684 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 3/3
138/138 [==============================] - 2s 14ms/step - loss: 0.1411 - auc_2: 0.5000 - binary_accuracy: 0.9684 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
[1] Evaluating hparams: {'activation': 'relu', 

InvalidArgumentError: Graph execution error:

Detected at node 'model_8/g_convo_layer_8/mul_1' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/olga/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/olga/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/olga/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/olga/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_once()
    F

In [ ]:
from ogb.nodeproppred import Evaluator
from ogb.graphproppred import Evaluator
evaluator = Evaluator(name = 'ogbg-molhiv')
evaluator


In [ ]:
node_features = GraphPropPredDataset(name='ogbg-molhiv')
test_data = b.make_tf_datasets(node_features)['test']


In [ ]:
labels = g.getLabels(test_data)
labels 

len(labels)


4113

In [ ]:
# problem: Ich habe eine letzte Frage: Ich muss um alle metriken zu tracken, die ich in config aufzähle für die jeweiligen datasets meine funktion create_model umschreiben und 'dataset_name' als parameter zusätzlich übergeben und dann config(dataset_name) aufrufen. Und create_grid_models ruft dann nochmal beide methoden auf. Es kommt mir wie richtig schlechter stil vor (normalerweise ist das ja unterwünscht, alles soll so modular wie möglich sein und die 'komplexeste' methode ruft dann alle helper auf, aber ich sehe hier keinen besseren weg, da metriken nur an model.compile übergeben werden können um dann so schön alle auf einmal getrackt zu werden. Was würdest du machen ?


# lösung: ja, den dataset_name weißt du ja wenn du die grid_search startest. die grid_search funktion muss also nur einmal die config berechnen und kann diese config dann an create_model übergeben. mit "das evaluierte dataset" meinte ich jetzt nicht speziell den aufruf einer bestimmten evaluate methode, sondern den generellen experimentellen auswertungsprozess für ein dataset (all inclusive, grid search etc.)




In [ ]:
g.config('ogbg-molhiv')


{'include_mask': False,
 'num_tasks': 1,
 'loss': <keras.losses.BinaryCrossentropy at 0x7f9ab4755e70>,
 'metrics': [keras.metrics.metrics.AUC,
  keras.metrics.metrics.BinaryAccuracy,
  keras.metrics.metrics.Precision,
  keras.metrics.metrics.Recall],
 'evaluator': <ogb.graphproppred.evaluate.Evaluator at 0x7f9ab4754b20>}